In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns


from sklearn.datasets import make_blobs
from sklearn.metrics import f1_score,accuracy_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report
from sklearn.metrics import matthews_corrcoef,cohen_kappa_score,balanced_accuracy_score


import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from catboost import CatBoostClassifier

from modAL.models import ActiveLearner, Committee
from modAL.uncertainty import uncertainty_sampling,margin_sampling,entropy_sampling
from functools import partial
from modAL.batch import uncertainty_batch_sampling

import warnings

warnings.filterwarnings('ignore')

In [3]:
#def compute ratio
def comp_ratio(data):
    instance = data[(data['label']==1)]
    rate=len(instance)/len(data)*100
    print(rate)
    return rate

# Splite Dataset

In [34]:
train_set=pd.read_csv('./randomtrain/3901890.csv') #3901890 6903102
test_set=pd.read_csv('./randomtest/3901890.csv')
len(train_set),len(test_set)

(124622, 53497)

In [35]:
#train_set2,test_set2=getLastSplit(data,0.3,dirty_ratio)

In [36]:
comp_ratio(train_set)
comp_ratio(test_set)

29.35918216687262
29.364263416640185


29.364263416640185

# Random

In [37]:
x,y=train_set.shape
#pool 
X_Pool = train_set.iloc[:,0:y-1].values
y_Pool = train_set.iloc[:,y-1].values

In [38]:
#test set
X_test=test_set.iloc[:,0:y-1]
y_tru=test_set.iloc[:,y-1]
print(len(X_test),len(y_tru))

53497 53497


In [39]:
#Pool
def initial_random(n_initial,X_Pool,y_Pool):
    #inital
    initial_idx = np.random.choice(range(len(X_Pool)), size=n_initial, replace=False)
    #initial Labeled data
    #X_initial, y_initial = X_train[], y_train[initial_idx]
    X_L = X_Pool[initial_idx]
    y_L = y_Pool[initial_idx]
    # Unlabeled data
    # X_U = Pool_X[ini_num:]
    # y_U = Pool_X[ini_num:]
    X_U, y_U = np.delete(X_Pool, initial_idx, axis=0), np.delete(y_Pool, initial_idx, axis=0)
    return X_L,y_L,X_U,y_U

In [44]:
# Set RNG seed for reproducibility.
RANDOM_STATE_SEED = 84
np.random.seed(RANDOM_STATE_SEED)

n_initial = 50
N_QUERIES = 200

In [45]:
X_initial,y_initial,X_re,y_re=initial_random(n_initial,X_Pool,y_Pool)
df_X_i = pd.DataFrame(X_initial)
df_y_i = pd.DataFrame(y_initial)
df_X_r = pd.DataFrame(X_re)
df_y_r = pd.DataFrame(y_re)


df_X_i.to_csv('./randomtrain/3901890_X_initial_05.csv', index=False)
df_y_i.to_csv('./randomtrain/3901890_y_initial_05.csv', index=False)
df_X_r.to_csv('./randomtrain/3901890_X_re_05.csv', index=False)
df_y_r.to_csv('./randomtrain/3901890_y_re_05.csv', index=False)

# X_initial=pd.read_csv('./randomtrain/6903102_X_initial.csv').values
# y_initial=pd.read_csv('./randomtrain/6903102_y_initial.csv').values.ravel()
# X_re=pd.read_csv('./randomtrain/6903102_X_re.csv').values
# y_re=pd.read_csv('./randomtrain/6903102_y_re.csv').values.ravel()
print(X_re.shape,X_initial.shape)
print(y_re.shape,y_initial.shape)

(124572, 6) (50, 6)
(124572,) (50,)


# Unsupervised

In [14]:
#Pool
def initial_un(n_initial,X_Pool,y_Pool):
    #initial Labeled data
    X_L = X_Pool[:n_initial]
    y_L = y_Pool[:n_initial]
    # Unlabeled data
    X_U = X_Pool[n_initial:]
    y_U = y_Pool[n_initial:]
    return X_L,y_L,X_U,y_U

## iforest

In [17]:
train_data_auto=pd.read_csv("./other/oc_score_low_top_seadata.csv")

In [18]:
# pool 
X_Pool = train_data_auto.iloc[:,0:y-1].values
y_Pool = train_data_auto.iloc[:,y-1].values

In [19]:
X_Pool

array([[ 1.32432000e+05, -1.09772984e+00, -4.77683642e-01,
        -5.17109417e-01, -3.81173064e-01,  6.93024208e-01],
       [ 4.20860000e+04, -1.09780278e+00, -4.78415494e-01,
        -5.09796307e-01, -3.81173064e-01,  6.93024208e-01],
       [ 6.37400000e+03, -4.56215697e-01, -4.99556132e-01,
        -1.21803449e+00, -3.81173064e-01,  6.93024208e-01],
       ...,
       [ 1.64090000e+04, -4.47973389e-01, -4.78309177e-01,
        -1.76842072e-01, -9.21040942e-02, -2.16041849e-01],
       [ 1.45389000e+05, -4.48119271e-01, -4.76555033e-01,
        -1.05320423e-01, -9.21040942e-02, -2.16041849e-01],
       [ 8.31650000e+04, -4.48119271e-01, -4.76620553e-01,
        -1.06167501e-01, -9.21040942e-02, -2.16041849e-01]])

In [ ]:
X_in_au,y_in_au,X_re_au,y_re_au=initial_data(n_initial,X_Pool,y_Pool)
print(len(X_in_au),len(X_re_au))